In [10]:
creds = {
    "base_url": 'https://10.10.20.90', "username": 'admin', "password": 'C1sco12345'
}

deviceId = {"deviceId": '10.10.1.11'}

In [11]:
from cisco_sdwan.base.rest_api import Rest

with Rest(**creds) as api:
    print(api.server_version)

20.10.1


In [17]:
import json
from cisco_sdwan.base.rest_api import Rest

def print_json(j_obj):
    print(json.dumps(j_obj, indent=2))

with Rest(**creds) as api:
    reply = api.get('device/control/connections', **deviceId)
    print_json(reply)

{
  "header": {
    "generatedOn": 1683582021511,
    "viewKeys": {
      "uniqueKey": [
        "vdevice-dataKey"
      ],
      "preferenceKey": "grid-ControlConnection"
    },
    "columns": [
      {
        "title": "Instance ID",
        "property": "instance",
        "width": 100,
        "dataType": "number"
      },
      {
        "title": "Peer Type",
        "property": "peer-type",
        "hideable": false,
        "width": 100,
        "dataType": "string"
      },
      {
        "title": "Peer Protocol",
        "property": "protocol",
        "width": 150,
        "dataType": "string"
      },
      {
        "title": "Peer System IP",
        "property": "system-ip",
        "hideable": false,
        "width": 200,
        "dataType": "ip"
      },
      {
        "title": "Site ID",
        "property": "site-id",
        "width": 100,
        "dataType": "number"
      },
      {
        "title": "Domain ID",
        "property": "domain-id",
        "width": 100,
 

In [18]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

with Rest(**creds) as api:
    control_connections = DeviceControlConnections.get(api, **deviceId)
    print(control_connections)

[
  {
    "domain-id": 1,
    "instance": 0,
    "vdevice-name": "10.10.1.11",
    "behind-proxy": "No",
    "system-ip": "10.10.1.5",
    "remote-color": "default",
    "site-id": 101,
    "private-port": 12446,
    "controller-group-id": 0,
    "vdevice-host-name": "dc-cedge01",
    "local-color": "mpls",
    "uptime": "0:00:10:29",
    "peer-type": "vsmart",
    "shared-region-id-set": "N/A",
    "v-org-name": "DevNet Sandbox",
    "protocol": "dtls",
    "vdevice-dataKey": "10.10.1.11-0-mpls-10.10.1.5-default",
    "public-ip": "10.10.20.70",
    "public-port": 12446,
    "lastupdated": 1683582040394,
    "state": "up",
    "private-ip": "10.10.20.70",
    "uptime-date": 1683581400000
  },
  {
    "domain-id": 1,
    "instance": 0,
    "vdevice-name": "10.10.1.11",
    "behind-proxy": "No",
    "system-ip": "10.10.1.5",
    "remote-color": "default",
    "site-id": 101,
    "private-port": 12446,
    "controller-group-id": 0,
    "vdevice-host-name": "dc-cedge01",
    "local-color"

In [ ]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

with Rest(**creds) as api:
    control_connections = DeviceControlConnections.get(api, deviceId='192.168.250.168')
    print(control_connections.field_names)

In [ ]:
print(control_connections.field_info('site_id'))

In [ ]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

with Rest(**creds) as api:
    control_connections = DeviceControlConnections.get(api, deviceId='192.168.250.168')

    header = control_connections.field_info('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date')
    print(f"{header[0]:14} {header[1]:12} {header[2]:12} {header[3]:5} {header[4]}")
    for row in control_connections.field_value_iter('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date'):
        print(f"{row.system_ip:14} {row.local_color:12} {row.remote_color:12} {row.state:5} {row.uptime_date}")

In [ ]:
from datetime import datetime, timezone
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

def datetime_format(timestamp):
    if timestamp is None:
        return ""

    dt = datetime.fromtimestamp(float(timestamp) / 1000, tz=timezone.utc)
    return f"{dt:%Y-%m-%d %H:%M:%S %Z}"

with Rest(**creds) as api:
    control_connections = DeviceControlConnections.get(api, deviceId='192.168.250.168')

    header = control_connections.field_info('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date')
    print(f"{header[0]:14} {header[1]:12} {header[2]:12} {header[3]:5} {header[4]}")
    for row in control_connections.field_value_iter('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date', uptime_date=datetime_format):
        print(f"{row.system_ip:14} {row.local_color:12} {row.remote_color:12} {row.state:5} {row.uptime_date}")

In [ ]:
import logging
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.tasks.implementation import TaskBackup, BackupArgs

# Setup logging to visualize progress
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")

# Equivalent to 'sdwan backup all --no-rollover --save-running'
task_args = BackupArgs(
    save_running = True,
    no_rollover = True,
    workdir = 'backup_test',
    tags = ['all']
)

with Rest(**creds) as api:
    task = TaskBackup()
    task_output = task.runner(task_args, api)

    if task_output:
        print('\n\n'.join(str(entry) for entry in task_output))

    task.log_info(f'Task completed {task.outcome("successfully", "with caveats: {tally}")}')

In [ ]:
import logging
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.tasks.implementation import TaskRestore, RestoreArgs

# Setup logging to visualize progress
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")

# Equivalent to 'sdwan restore all --workdir backup_test'
task_args = RestoreArgs(
    workdir = 'backup_test',
    tags = 'all'
)

with Rest(**creds) as api:
    task = TaskRestore()
    task_output = task.runner(task_args, api)

    if task_output:
        print('\n\n'.join(str(entry) for entry in task_output))

    task.log_info(f'Task completed {task.outcome("successfully", "with caveats: {tally}")}')